# Core Elements of LangGraph

LangGraph is a library for building stateful, multi-actor applications with LLMs. It extends LangChain by allowing you to create cyclic graphs, which are essential for agentic workflows.

### The Restaurant Analogy 🍽️
- **State**: The Order Ticket. Everyone (Chef, Waiter, Dishwasher) reads and updates it.
- **Nodes**: The Staff. The waiter takes the order, the chef cooks the meal.
- **Edges**: The Workflow. After the chef cooks, the ticket goes to the waiter.
- **Conditional Edges**: The Decision. If the meal is ready, go to the customer; if not, wait.
- **Checkpointer**: The Log Book. If the restaurant closes and reopens, the log book remembers where every order was left.

## 1. State
The `State` is the source of truth for your graph. It is a shared data structure defined by a schema (TypedDict or Pydantic).

In [ ]:
from typing import TypedDict, Annotated, List
from pydantic import BaseModel, Field
import operator

# Option A: TypedDict (Standard)
class GraphState(TypedDict):
    messages: Annotated[List[str], operator.add]  # Appends to list instead of overwriting
    count: int

# Option B: Pydantic (Better for validation)
class PydanticState(BaseModel):
    query: str = Field(description="User input query")
    history: List[str] = Field(default_factory=list)

## 2. Nodes
Nodes are Python functions that take the current `State` and return a dictionary that *updates* the state.

In [ ]:
def chatbot_node(state: GraphState):
    # Logic here (e.g., call an LLM)
    new_message = "Hello! How can I help?"
    return {"messages": [new_message], "count": state.get("count", 0) + 1}

## 3. Edges & Navigation
Edges define how to move from one node to another.

In [ ]:
from langgraph.graph import StateGraph, START, END

workflow = StateGraph(GraphState)
workflow.add_node("agent", chatbot_node)

# 1. Static Edge: Always go from START to 'agent'
workflow.add_edge(START, "agent")

# 2. Conditional Edge: Decide where to go next based on state
def should_continue(state: GraphState):
    if state["count"] > 5: return "finish"
    return "continue"

workflow.add_conditional_edges(
    "agent", 
    should_continue, 
    {"finish": END, "continue": "agent"}
)

## 4. Command (Modern Control Flow)
In newer versions of LangGraph, you can use the `Command` object to handle both **state updates** and **navigation** directly from within a node, often eliminating the need for separate conditional edges.

In [ ]:
from langgraph.types import Command

def smart_node(state: GraphState):
    # Update state AND tell the graph where to go next
    return Command(
        update={"messages": ["Thinking..."]},
        goto="agent" if state["count"] < 3 else END
    )

## 5. Persistence & Checkpointers
LangGraph can save the state of the graph after every step. This allows for "memory" across different sessions and error recovery.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

# Using a thread_id to identify a unique conversation
config = {"configurable": {"thread_id": "user_123"}}
app.invoke({"messages": ["Hi"], "count": 0}, config)

## 6. Breakpoints (Human-in-the-loop)
Breakpoints allow you to pause the graph execution before or after a specific node to wait for human approval or input.

In [ ]:
# Compile the app with a breakpoint before the 'agent' node
app = workflow.compile(checkpointer=memory, interrupt_before=["agent"])

# Execution will STOP here and wait for a command to resume.

## 7. Subgraphs
You can use a compiled graph as a node inside another graph. This is great for modularizing complex agents.

In [ ]:
inner_workflow = StateGraph(GraphState)
# ... define nodes/edges ...
inner_app = inner_workflow.compile()

parent_workflow = StateGraph(GraphState)
parent_workflow.add_node("subgraph", inner_app)
parent_workflow.add_edge(START, "subgraph")

## 8. Tools & ToolNode
Agents often need to use tools. LangGraph provides a prebuilt `ToolNode` to handle the execution of tool calls generated by models.

In [ ]:
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool
def get_weather(city: str):
    """Get the current weather for a city."""
    return f"The weather in {city} is sunny."

tools = [get_weather]
tool_node = ToolNode(tools)

# Add it to your graph like any other node
workflow.add_node("tools", tool_node)

## 9. Prebuilt Graphs (`create_react_agent`)
For standard designs like a ReAct agent, LangGraph provides high-level factories to create them in one line.

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o")
agent = create_react_agent(model, tools, checkpointer=memory)

## 10. Message Types
Conversations are usually stored as a list of message objects.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage

chat_history = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?"),
    AIMessage(content="Paris.")
]